In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,spark-be88605473ed44abbde1a98d0af32961,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=k8s://https://kubernetes.default.svc.cluster.local:443 appName=livy-session-0>

In [5]:
raw_sim_ft = spark.read.format("csv").option("header", "true").option("sep", ",").load("s3a://databeanstalk-community/data/safegraph_patterns_simulated__1_-91d51.csv")
raw_sim_ft.createOrReplaceTempView("safegraph_sim_foot_traffic")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
safegraph_patterns = spark.sql("""
select x.*, INT(YEAR(FROM_UNIXTIME(date_range_start))) as year, 
                                        INT(MONTH(FROM_UNIXTIME(date_range_start))) as month, 
                                       case when region in ('NY', 'PA', 'NJ') then 'NYC MSA' else 'US' end msa, location_name
from safegraph_sim_foot_traffic x""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Function to extract ARRAY or JSON columns for deeper analysis
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json

def parser(element):
  return json.loads(element)

def parser_maptype(element):
  return json.loads(element, MapType(StringType(), IntegerType()))

jsonudf = udf(parser, MapType(StringType(), IntegerType()))

convert_array_to_dict_udf = udf(lambda arr: {idx: x for idx, x in enumerate(json.loads(arr))}, MapType(StringType(), IntegerType()))

def explode_json_column_with_labels(df_parsed, column_to_explode, key_col="key", value_col="value"):
  df_exploded = df_parsed.select("safegraph_place_id", "location_name", "msa", "date_range_start", "year", "month", "date_range_end", explode(column_to_explode)).selectExpr("safegraph_place_id", "date_range_end", "location_name","msa", "date_range_start", "year", "month", "key as {0}".format(key_col), "value as {0}".format(value_col))
  return(df_exploded)

def explode_safegraph_json_column(df, column_to_explode, key_col="key", value_col="value"):
  df_parsed = df.withColumn("parsed_"+column_to_explode, jsonudf(column_to_explode))
  df_exploded = explode_json_column_with_labels(df_parsed, "parsed_"+column_to_explode, key_col=key_col, value_col=value_col)
  return(df_exploded)

def explode_safegraph_array_colum(df, column_to_explode, key_col="index", value_col="value"):
  df_prepped = df.select("safegraph_place_id", "location_name", "msa", "date_range_start", "year", "month", "date_range_end", column_to_explode).withColumn(column_to_explode+"_dict", convert_array_to_dict_udf(column_to_explode))
  df_exploded = explode_json_column_with_labels(df_prepped, column_to_explode=column_to_explode+"_dict", key_col=key_col, value_col=value_col)
  return(df_exploded)

def explode_safegraph_visits_by_day_column(df, column_to_explode, key_col="index", value_col="value"):
  df_exploded = explode_safegraph_array_colum(df, column_to_explode, key_col=key_col, value_col=value_col)
  df_exploded = df_exploded.withColumn(key_col, col(key_col) + 1) # 1-indexed instead of 0-indexed
  return(df_exploded)


visits_by_day = explode_safegraph_visits_by_day_column(safegraph_patterns, column_to_explode="visits_by_day", key_col="day", value_col="num_visits")
print(visits_by_day.count())
visits_by_day.createOrReplaceTempView('visits_by_day')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

408

In [8]:
%%sql 
create or replace temp view ft_raw as select * from visits_by_day

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%%sql
select * from ft_raw with limit 5;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%sql
create or replace temp view dunkin_foot_traffic  as select * from ft_raw where location_name = "Dunkin'"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [15]:
%%sql
select * from dunkin_foot_traffic with limit 5;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [17]:
pdf = spark.sql("select * from (select cast(year as integer) year, cast(month as integer) month, cast(day as integer) day, sum(num_visits) num_vists from dunkin_foot_traffic where msa = 'NYC MSA' and cast(concat(year, '-', lpad(cast(month as integer), 2, '0'), '-', lpad(cast(day as integer), 2, '0')) as date) <= '2020-01-12' group by cast(year as integer), cast(month as integer), cast(day as integer)) foo where year is not null").toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
all_nyc_dunkin = spark.sql("select * from (select cast(year as integer) year, cast(month as integer) month, cast(day as integer) day, sum(num_visits) num_vists from dunkin_foot_traffic where msa = 'NYC MSA' and cast(concat(year, '-', lpad(cast(month as integer), 2, '0'), '-', lpad(cast(day as integer), 2, '0')) as date) <= '2020-01-12' group by cast(year as integer), cast(month as integer), cast(day as integer)) foo where year is not null").toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
actual = spark.sql("select * from (select cast(year as integer) year, cast(month as integer) month, cast(day as integer) day, sum(num_visits) num_vists from dunkin_foot_traffic where msa = 'NYC MSA' and cast(concat(year, '-', lpad(cast(month as integer), 2, '0'), '-', lpad(cast(day as integer), 2, '0')) as date) > '2020-01-12'  group by cast(year as integer), cast(month as integer), cast(day as integer)) foo where year is not null").toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
all_nyc_dunkin_actual = spark.sql("select * from (select cast(year as integer) year, cast(month as integer) month, cast(day as integer) day, sum(num_visits) num_vists from dunkin_foot_traffic where msa = 'NYC MSA' and cast(concat(year, '-', lpad(cast(month as integer), 2, '0'), '-', lpad(cast(day as integer), 2, '0')) as date) > '2020-01-12'  group by cast(year as integer), cast(month as integer), cast(day as integer)) foo where year is not null").toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
import pandas as pd 
pdf['date'] = pd.to_datetime(pdf[["year", "month", "day"]])
all_nyc_dunkin['date'] = pd.to_datetime(all_nyc_dunkin[["year", "month", "day"]])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
pdf = pdf.sort_values('date')
all_nyc_dunkin = all_nyc_dunkin.sort_values('date')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
pdf['ds'] = pdf['date']
pdf = pdf.sort_values('date')
pdf['y'] = pdf['num_vists']

all_nyc_dunkin['ds'] = all_nyc_dunkin['date']
all_nyc_dunkin = all_nyc_dunkin.sort_values('date')
all_nyc_dunkin['y'] = all_nyc_dunkin['num_vists']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j").setLevel(logging.ERROR)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…